In [2]:
from selenium import webdriver
import yfinance as yf
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
ticker = 'BBAS3.SA'
acao = yf.Ticker(ticker)
info = acao.info


In [4]:
# 1. Coletar Dados do BBAS3
ticker = 'BBAS3.SA'
data = yf.download(ticker, start='2010-01-01', end='2023-01-20')
data

 

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,BBAS3.SA,BBAS3.SA,BBAS3.SA,BBAS3.SA,BBAS3.SA,BBAS3.SA
Date,,,,,,
2010-01-04,5.466509,14.950000,15.000000,14.850,14.900,7249400
2010-01-05,5.411664,14.800000,15.050000,14.595,15.050,6129800
2010-01-06,5.418976,14.820000,15.000000,14.710,14.745,5466200
2010-01-07,5.420805,14.825000,14.855000,14.705,14.720,3084400
2010-01-08,5.451886,14.910000,14.910000,14.735,14.850,4077800
...,...,...,...,...,...,...
2023-01-13,15.499995,17.825001,17.924999,17.475,17.750,20087000
2023-01-16,15.482600,17.805000,18.059999,17.660,17.830,18437800


In [14]:
historico = acao.history(period='max')
historico

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2000-01-03 00:00:00-02:00,0.495216,0.504649,0.483425,0.483425,1029600,0.000000,0.0
2000-01-04 00:00:00-02:00,0.483425,0.484605,0.460433,0.460433,628200,0.000000,0.0
2000-01-05 00:00:00-02:00,0.459844,0.465739,0.443336,0.465150,957600,0.000000,0.0
2000-01-06 00:00:00-02:00,0.465150,0.471634,0.459844,0.471634,410400,0.000000,0.0
2000-01-07 00:00:00-02:00,0.471635,0.471635,0.461023,0.461023,788400,0.000000,0.0
...,...,...,...,...,...,...,...
2024-11-22 00:00:00-03:00,24.982139,25.119513,24.746644,25.119513,13737200,0.000000,0.0
2024-11-25 00:00:00-03:00,25.305947,25.355008,25.119513,25.266697,36581800,0.000000,0.0
2024-11-26 00:00:00-03:00,25.360001,25.780001,25.209999,25.670000,22044000,0.483304,0.0


In [6]:
dividendos = acao.dividends
historico_com_dividendos = historico[historico.index.isin(dividendos.index)]
historico_com_dividendos['DY'] =  (historico_com_dividendos['Dividends'] / historico_com_dividendos['Open'])*100
historico_com_dividendos.head(60)

/tmp/ipykernel_3063437/879929311.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historico_com_dividendos['DY'] =  (historico_com_dividendos['Dividends'] / historico_com_dividendos['Open'])*100


,Open,High,Low,Close,Volume,Dividends,Stock Splits,DY
Date,,,,,,,,
2014-12-12 00:00:00-02:00,6.469213,6.516761,6.136374,6.136374,14275200,0.169975,0.0,2.627445
2015-02-20 00:00:00-02:00,6.490881,6.490881,6.297322,6.355655,5348200,0.045600,0.0,0.702524
2015-03-24 00:00:00-03:00,6.411457,6.451866,6.274069,6.338722,8084600,0.188496,0.0,2.939987
2015-05-22 00:00:00-03:00,6.558528,6.580481,6.407600,6.407600,16918400,0.225569,0.0,3.439323
2015-06-12 00:00:00-03:00,6.305722,6.386315,6.252919,6.252919,13196600,0.144947,0.0,2.298658
2015-08-24 00:00:00-03:00,4.872617,5.177855,4.794207,5.021036,24489600,0.070520,0.0,1.447272
2015-09-14 00:00:00-03:00,4.637010,4.881812,4.617084,4.861886,17450400,0.133039,0.0,2.869069
2015-11-24 00:00:00-02:00,5.257521,5.386805,5.174206,5.343710,11256800,0.085403,0.0,1.624397
2015-12-14 00:00:00-02:00,5.066253,5.072090,4.850296,4.856132,16431200,0.137246,0.0,2.709024


In [7]:

for ano in range(dividendos.index[0].year, dividendos.index[-1].year + 1):
    # Filtra os dividendos do ano específico
    dividendos_ano = dividendos[dividendos.index.year == ano].sum()
    
    # Obtém o preço de fechamento no início do ano
    preco_inicio_ano = historico.loc[f'{ano}-01-01':'{ano}-01-31']['Close'].iloc[0]

    # Calcula o dividend yield anual
    dividend_yield_anual = (dividendos_ano / preco_inicio_ano) * 100

# Exibindo o dividend yield anual
for ano, yield_ in dividend_yield_anual.items():
    print(f"Dividend Yield de {ano}: {yield_:.2f}%")

TypeError: cannot do slice indexing on DatetimeIndex with these indexers [{ano}-01-31] of type str

In [ ]:
url = 'https://api.mziq.com/mzfilemanager/v2/d/5760dff3-15e1-4962-9e81-322a0b3d0bbd/5b90cdd9-0fb8-aaab-afbb-10ee2a4933ed?origin=1'
response = requests.get(url)
print(response)